# **Reti Neurali** 

## **Esercizio 1: Download e pre-processamento dei dati.**

Scaricare e pre-processare i dati per il successivo addestramento del modello. 

Il dataset che utilizzeremo sarà CIFAR10, scaricabile dalla libreria `tensorflow.keras.datasets`

In [16]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10

# Download dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

y_train = y_train.ravel()
y_test = y_test.ravel()

# Stampare le shape dei dati
print (x_train.shape)
print (y_train.shape)
print (x_test.shape)
print (y_test.shape)

(50000, 32, 32, 3)
(50000,)
(10000, 32, 32, 3)
(10000,)


In [17]:
from sklearn.preprocessing import StandardScaler

# Pre-processamento dei dati
# svolgimento...

x_train_flat = x_train.reshape((x_train.shape[0], -1)) # Appiattiamo le immagini
x_test_flat = x_test.reshape((x_test.shape[0], -1))

# Standardizzazione
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train_flat)
x_test_scaled = scaler.transform(x_test_flat)


#stampo i dati scalati 
print(x_train_scaled.shape)
print(x_test_scaled.shape)
print(x_train_flat.shape)
print(x_test_flat.shape)

(50000, 3072)
(10000, 3072)
(50000, 3072)
(10000, 3072)


## **Esercizio 2: Creare modello MLP**

Per creare il modello MLP utilizziamo l' oggetto `MLPClassifier` dal modulo `sklearn.neural_networks`. Questo è un oggetto molto complesso che prevede la possibilità di specificare tanti parametri, permettendoci una personalizzazione molto dettagliata. Vediamo di seguito gli argomenti principali:

- `hidden_layer_sizes`: rappresenta la struttura dell' MLP, sotto forma di una tupla. La tupla deve essere composta da numeri interi, ogni numero indica il numero di neuroni presenti nel rispettivo layer.

Esempio: 

`hidden_layer_sizes` = `(100)`

creerà un solo layer con 100 neuroni

`hidden_layer_sizes` = `(100, 50)`

creerà due layer, il primo con 100 neuroni, il secondo invece con 50.

- `max_iter`: massimo numero di iterazioni per raggiungere la convergenza. 

- `activation`: indica quale funzione di attivazione utilizzare, valori possibili sono `'relu'`, `'logistic'`, `'tanh'` and `'identity'`.

- `solver`: indica quale algoritmo di ottimizzazione utilizzare, valori possibili sono `'adam'`, `'sgd'` and `'lbfgs'`.

- `learning_rate_init`: valore iniziale del learning rate.

- `verbose`: valore booleano che, se impostato su `True`, stampa l' output di ogni iterazione di training. Molto utile per monitorare il training.

- `random_state`: fissa il seed della randomizzazione.


Per iniziare creiamo un MLP molto basilare, alleniamolo e testiamone le performance. Come parametri utilizzeremo:

- `hidden_layer_sizes` = `(100)`

- `max_iter` = `20`

- `random_state` = `42`

In [18]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Creare MLP 
# svolgimento...
mlp = MLPClassifier(hidden_layer_sizes=(100), max_iter=20, random_state=42)

# Allenare MLP
# svolgimento...
mlp.fit(x_train_scaled, y_train)

# Valutare MLP
# svolgimento...

y_pred = mlp.predict(x_test_scaled)


# Stampare l' accuratezza
# svolgimento...
accuracy = accuracy_score(y_test, y_pred) 
print(f"Accuratezza sul test set: {accuracy:.4f}")

Accuratezza sul test set: 0.5057


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


## **Esercizio 2.1: Aumentiamo i parametri del nostro modello**

Proviamo adesso ad aumentare i dettagli del nostro modello, modificando o aggiungendo i parametri sopra specificati. 

In [19]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Creare MLP con più strati e altre specifiche
# svolgimento... 

# Creare MLP 
mlp = MLPClassifier(hidden_layer_sizes=(100), activation='identity', solver= 'lbfgs', learning_rate_init= 0.001, verbose= True, max_iter=20, random_state=42)

# Allenare MLP
mlp.fit(x_train_scaled, y_train)

# Valutare MLP
y_pred = mlp.predict(x_test_scaled)

# Stampare l' accuratezza
accuracy = accuracy_score(y_test, y_pred) 
print({accuracy})

{0.3679}


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


## **Esercizio 3: Implementare manualmente l' algoritmo di early stopping.**

L' algoritmo di early stopping ci permette di terminare anticipatamente l' allenamento di un modello nel caso in cui questo raggiunga la convergenza. Supponiamo infatti che il nostro modello raggiunga un certo livello di accuratezza e che non riesca a migliorare oltre quel livello. Questo significa che il modello, da quel momento in poi, non sta più apprendendo nuove informazioni, per cui le successive iterazioni sono superflue, ed inoltre rischiano di essere dannose, spingendo il modello verso l' overfitting. 

L' early stopping verifica ad ogni iterazione che l' accuratezza del modello sia incrementata di una certa tolleranza. Se questa tolleranza non viene superata per un certo numero di epoche, allora possiamo decidere di stoppare l' allenamento in quanto il modello ha raggiunto la convergenza.

**N.B: per applicare early stopping è necessario specificare i seguenti parametri dell' MLP:**

- `warm_start`=`True` in modo che il training proceda dallo stato attuale del modello e non dall' inizializzazione.

- `max_iter`=`1` in modo che il modello venga allenato per una sola epoca. Per l' early stopping infatti dovremo gestire manualmente il numero di iterazioni.

In [23]:
import numpy as np
from sklearn.neural_network import MLPClassifier 
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split

x_train_partial, x_val, y_train_partial, y_val = train_test_split(x_train_scaled, y_train, test_size=0.2, random_state=42)

n_total_epochs = 100  
patience = 10         
tolerance = 1e-4      

best_val_accuracy = 0.0
epochs_without_improvement = 0

mlp = MLPClassifier(hidden_layer_sizes=(100), warm_start=True, max_iter=1, random_state=42)

for epoch in range(n_total_epochs):
    mlp.fit(x_train_partial, y_train_partial)
    
    y_val_pred = mlp.predict(x_val)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    print(f"Epoch {epoch+1}, Validation Accuracy: {val_accuracy:.4f}")

    if val_accuracy > best_val_accuracy + tolerance:
        best_val_accuracy = val_accuracy
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1
    
    if epochs_without_improvement >= patience:
        print(f"Early stopping si ferma all'epoca {epoch+1}")
        break

y_test_pred = mlp.predict(x_test_scaled)
final_test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Accuratezza finale sul test set: {final_test_accuracy:.4f}")

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(


Epoch 1, Validation Accuracy: 0.4290
Epoch 2, Validation Accuracy: 0.4517
Epoch 3, Validation Accuracy: 0.4555
Epoch 4, Validation Accuracy: 0.4604
Epoch 5, Validation Accuracy: 0.4681
Epoch 6, Validation Accuracy: 0.4732
Epoch 7, Validation Accuracy: 0.4734
Epoch 8, Validation Accuracy: 0.4735
Epoch 9, Validation Accuracy: 0.4774
Epoch 10, Validation Accuracy: 0.4781
Epoch 11, Validation Accuracy: 0.4783
Epoch 12, Validation Accuracy: 0.4772
Epoch 13, Validation Accuracy: 0.4781
Epoch 14, Validation Accuracy: 0.4757
Epoch 15, Validation Accuracy: 0.4786
Epoch 16, Validation Accuracy: 0.4766
Epoch 17, Validation Accuracy: 0.4743
Epoch 18, Validation Accuracy: 0.4742
Epoch 19, Validation Accuracy: 0.4696
Epoch 20, Validation Accuracy: 0.4685
Epoch 21, Validation Accuracy: 0.4672
Epoch 22, Validation Accuracy: 0.4659
Epoch 23, Validation Accuracy: 0.4621
Epoch 24, Validation Accuracy: 0.4620
Epoch 25, Validation Accuracy: 0.4604
Early stopping si ferma all'epoca 25
Accuratezza finale sul